# The `task` statement and option `-q`

* **Difficulty level**: intermediate
* **Time need to lean**: 30 minutes or less
* **Key points**:
   * Tasks are defined by `task` statement as part of substeps
   * Tasks are executed independently of SoS process and can be executed on remote hosts, even if the host does not share file systems with local host

## The concept of task

<div class="bs-callout bs-callout-warning" role="alert">
    <p><b>Tasks</b> are part of SoS substeps that are encapsulated and executed independently of the main workflow, possibly on remote hosts and cluster systems.</p>  
</div>

The idea behind tasks is that a workflow can contain a variety of substeps (scripts etc). Some of them are light weight and can be executed together with the workflow (on a local laptop or desktop), and some can be computational or resource intensive and have to be executed on the cluster, or have to be executed on specific remote hosts due to unavailablity of software or data.

Whereas traditional workflow systems require all pieces of workflows to be executed on the same host, SoS allows you to send pieces of workflow to remote hosts, collect their results, and continue. This is achieved through a task mechanism.

## The `task` statement

Technically speaking, a **substep** consists of everything after the `input` statement. It can be repeated with subsets of input files or parameters defined by input options `group_by` or `for_each`. For example, if `bam_files` is a list of bam files,

```sos
[10]
input: bam_files, group_by=1
output: f"{_input}.bai"

print(f'Indexing {_input}')
run: expand=True
    samtools index {_input}
```

execute a shell script to process each bam file. This is done sequentially for each input file, and is performed by SoS.

You can easily specify part or all of a step process as **tasks**, by prepending the statements with a `task` keyword:


```sos
[10]
input: bam_files, group_by=1
output: f"{_input}.bai"

print(f'Indexing {_input}')

task:
run: expand=True
    samtools index {_input}
```

This statement declares the rest of the step process as a `task`. For each input file, a task will be created with an ID determined from task content and context (input and output files, variables etc). 

## Specification of task queue

<div class="bs-callout bs-callout-hint" role="alert">
    <h4>A <b>queue</b> is needed to execute a task</h4>
    <p>The <code>task</code> statement will be ignored if no task queue is specified, either from command line with option <code>-q</code> or option <code>queue</code> of the <code>task</code> statement.</p>  
</div>

### `task` statement is ignored by default

Tasks need to be submitted to a **task queue** to be executed. If you do not specify queue, the task statement will be ignored.

In [1]:
%run -v1

input: for_each=dict(i=range(2))
task:
sh: expand=True
  echo "I am task {i}"

I am task 0
I am task 1


### Executing tasks in a `localhost` queue

If you do not have anything defined in `~/.sos/hosts.yml` (see [host configuration](host_setup.html) for details), a `localhost` task queue will be available to you. Technically speaking this is a `process` queue with a maximum of 10 concurrent jobs. To submit the task to this queue, you can specify it as the `queue` option of `task` statement.

In [2]:
%run -v1

input: for_each=dict(i=range(2))
task: queue='localhost'
sh: expand=True
echo "I am task {i}"

As you can see, two tasks are generated from two substeps. The tasks have different IDs (because they have different value for variable `i`) and are executed successfully on `localhost`.

A more convenient way to specify task queue is to use the `-q` option of command `sos run` (or magic `%run`,  `%sosrun`, or `%runfile`) as follows:

In [3]:
%run -q localhost -v1

input: for_each=dict(i=range(2))
task:
sh: expand=True
echo "I am task {i}"

The differences between these two methods are that

1. Task queue specified by option `-q` is applied to all tasks defined in the workflow
2. If both `-q` and option `queue` are specified, task queue specified by option `quque` will be used for specific tasks.

That is to say, you should generally use `-q` to send tasks to different task queue, and use `queue` for tasks that are designed for specified queues.

Under the hood, two task files are created under `~/.sos/tasks` (e.g. `4fe780b5245b9e7b.task`) and SoS executes commands such as `sos execute 4fe780b5245b9e7b` to execute them, and collect result from the tasks. This is why you cannot see the output from the `echo "I am task {i}"` command executed by the tasks.

### Executing tasks on remote servers

<p align="center" height="500">
  <img src="https://vatlab.github.io/sos-docs/doc/media/remote_2_task.jpeg">
</p>

A `process` queue, like a localhost, execute the task directly, subject to the number of concurrent tasks. If you have a properly configured remote host (namely, have public-key authentication, have SoS installed, specified its `address`, `paths` etc in `hosts` configuration), you can send tasks to the remote host for execution.

More specifically, with the following example, SoS

1. Creates a task with an unique ID (`c600532cd99c02c2`) determined by the content of the task.
2. Copy the task file to remote host, translating `input` and `output` paths if two systems have different file systems.
3. Copy input files to remote host if needed.
4. Execute the task by executing something like
  ```
  ssh host "bash --login -c sos execute c600532cd99c02c2"
  ```
5. Monitor the status of the task and update the status in the notebook.
6. Copy the output file to local host if needed.

So with a host definition similar to the following

```yaml
hosts:
  bcb:
    address: bcbdesktop.mdanderson.edu
    max_running_jobs: 100
    paths:
        home: /Users/{user_name}/scratch
```

I can submit a task with a `R` script to be executed on `bcb`

In [4]:
%run -q bcb

output: 'test1.png'
task:
R:
  set.seed(1)
  x <- 1:100
  y <- -0.05*x + rnorm(100)
  png("test1.png", height=400, width=600)
  plot(x, y, pch=19, col=rgb(0.5, 0.5, 0.5, 0.5), cex=1.5)
  abline(lm(y ~ x))
  dev.off()
  

INFO: No matching tasks are identified. Use option -a to check all tasks.
INFO: c600532cd99c02c2 started


The output file is available locally after the completion of the task.

In [5]:
%preview test1.png

<div class="bs-callout bs-callout-hint" role="alert">
    <h4>Difference between options <code>-q</code> and <code>-r</code></h4>
    <p>If you have seen a similar example in <a href="remote_execution.html">remote execution of workflows</a>, you will find that option <code>-r</code> and <code>-q</code> are similar in that they send the R script to remote host for execution. There are however a few key differences:
<ul>
  <li>Option <code>-r</code> execute the entire workflows on remote host while option <code>-q</code> only part of substeps. In particular, option <code>-r</code> starts one `sos run` process on remote host and option <code>-q</code> starts two <code>sos execute</code> processes on the remote host.</li>
<li> Option <code>-r</code> uses input files on remote host and leaves output files on remote host. Option <code>-q</code> copies input files from local to remote host, and copies output files from remote to local host after the tasks are completed.</li>
    </ul>
    </div>

### Executing tasks on a cluster system

<p align="center" height="500">
  <img src="https://vatlab.github.io/sos-docs/doc/media/remote_3_tasks.jpeg">
</p>

A server can also be a `pbs` queue, which represent a batch job management system such as Slurm, IBM LSF, or Torque. With proper configuration (again, see [host configuration](host_setup.html) for details), you will be able to submit your tasks to a cluster system as cluster jobs. The cluster system can be a localhost if you are submitting tasks on the headnode of a cluster system or a remote host (if you are submitting tasks remotely.

For example, with a host configuration similar to


```yaml
hosts:
     htc:
        address: htc_cluster.mdanderson.edu
        description: HTC cluster (PBS)
        queue_type: pbs
        status_check_interval: 60
        submit_cmd: qsub {job_file}
        status_cmd: qstat {job_id}
        kill_cmd: qdel {job_id}        
        nodes: 1
        cores: 4
        walltime: 01:00:00
        mem: 4G
        task_template: |
            #!/bin/bash
            #PBS -N {job_name}
            #PBS -l nodes={nodes}:ppn={cores}
            #PBS -l walltime={walltime}
            #PBS -l mem={mem//10**9}GB
            #PBS -o ~/.sos/tasks/{task}.out
            #PBS -e ~/.sos/tasks/{task}.err
            #PBS -m n
            module load R
            {command}
```

You can submit your task to the cluster system

In [6]:
%run -q htc -s force

output: 'test.png'
task: walltime='1h', mem='2G', nodes=1, cores=1

R:
  set.seed(1)
  x <- 1:100
  y <- -0.03*x + rnorm(50)
  png("test.png", height=400, width=600)
  plot(x, y, pch=19, col=rgb(0.5, 0.5, 0.5, 0.5), cex=1.5)
  abline(lm(y ~ x))
  dev.off()

INFO: No matching tasks are identified. Use option -a to check all tasks.


Again, the output of the task will be synchronized to local host

In [7]:
%preview test.png

Although the usage of the workflow is largely the same as the previous example, the `task` statement has a few options to specify the resources used by the task, as required by the cluster system. The value of these options will be used to expand a template specified in the host specification file to generate a shell script that will be created and copied to remote host. The generated shell script could be found in the `-v4` output of the `sos status` or magic `%task status`

In [8]:
%task status 1fafb4489a7b4b47 -q htc -v4

Then, instead of command
```
ssh host "bash --login -c sos execute c600532cd99c02c2"
```
as in the last example, a command like
```
ssh host "bash --login -c qsub ~/.sos/tasks/1fafb4489a7b4b47.sh"
```
was used to submit the job to the cluster system.

The entire workflow for executing tasks on a remote cluster system contains five steps and can be depicted as follows. The process would be a lot simpler if you are allowed to execute workflows and submit tasks on the headnode of a cluster system since no file synchronization would be needed.

<p align="center">
  <img src="https://vatlab.github.io/sos-docs/doc/media/task_overview.png" width="80%">
</p>

## Task Options

The `task` statement accepts the following options

| Option | Usage | reference | 
| -- | -- | --  | 
| `shared` | Return variable to the substep | this tutorial |
| `env` | Customized environment | this tutorial |
| `prepend_path` | Customized `$PATH` | this tutorial |
| `walltime` | Total `walltime` of task | [task template](task_template.html) |
| `mem` | Memory required for task | [task template](task_template.html)  |
| `nodes` | Number of computing nodes | [task template](task_template.html)  |
| `cores` | Number of cores per node | [task template](task_template.html)  |
| `workdir` |  | [path translation and file synchronization ](task_files.html) |
| `to_host` | | [path translation and file synchronization ](task_files.html) |
| `from_host` | | [path translation and file synchronization ](task_files.html) |
| `map_vars` | | [path translation and file synchronization ](task_files.html) |
| ANY | Any template variable | [task template](task_template.html)  |

We will discuss some options in this tutorial, and leave the rest to specific tutorials.

### Option `shared`

SoS tasks are executed externally and by default does not return any value. Similar to the `shared` step option (that passes step variables to the workflow), you can use `shared` option to pass task variables to the step in which the task is defined.

For example, the following script perform some simulations in 10 tasks and return the result by variable `rng`, which is then shared to the workflow by step option `shared` so that it can be available to the next step.

In [9]:
%run -q localhost
[10 (simulate): shared='rng']
task: shared='rng'
import random
rng = random.randint(1, 1000)

[20]
print(rng)


INFO: 62f388838fadbc2f started
898


Also similar to step option `shared`, task option `shared` accepts a single variable (e.g. `rng`), a sequence of variables  (e.g. `('rng', 'sum')`), a dictionary of variable derived from an expression (e.g. `{'result': 'float(open(output).read())'}`, or sequences of names and variables. In the dictionary case, the values of the dictionary should be an expression (string), that will be evaluated upon the completion of the task, and assign to the specified variable.

Please refer to [sharing variables across steps (step and task option `shared` and target `sos_variable`)](doc/user_guide/shared_variables.html) for more details on variable sharing.

### Option `env`

The `env` option allow you to modify runtime environment, similar to the `env` parameter of the `subprocess.Popen` function. For example, you can set an environment variable `DEBUG` for the task as follows:

```sos
task:  env={'DEBUG': '1'}
run:
   mycommand 
```

### Option `prepend_path`

Option `prepend_path` prepend specified `paths` before `$PATH` before the task is executed. It provides a quick method to fix `$PATH` on a specific remote host.

```sos
task:  prepend_path='/path/to/mycommand'
run:
   mycommand 
```

If you have found yourself using this option a lot for a remote host, consider adding the paths to the `task_template` with commands such as

```sh
export PATH=/path/to/mycommand:$PATH
```
or using host-specific methods to set up environment properly, e.g.
```sh
module load R/3.4.4
```

## Further reading

* [Remote execution of workflow](remote_execution.html)
* [Sharing variables across steps (step and task option `shared` and target `sos_variable`)](doc/user_guide/shared_variables.html)